# House Price - Data Processing with TF.Transform

This notebook will report all the required code to create and export a data processing pipeline using Apache Beam and TF.Transform.

Whenever possible the notebook will be coded to support cloud run on GCP and local run.

## Set global parameters

In [1]:
PROJECT = 'project'         # change to your project_Id
BUCKET = 'houseprice_tftransform'     # change to your bucket name
REGION = 'region'                     # change to your region


LOCAL_ROOT_DIR = 'data'          # directory where the data are located is stored locally to support exploration
INPUT_FILE = "train.csv"

## Import required modules

In [2]:
import os
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION

In [3]:
# Import data processing libraries
import pandas as pd
import tensorflow as tf
# Import Tensorflow Transform
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import dataset_schema
# Import Apache Beam
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
# Python shutil module enables us to operate with file objects easily and without diving into file objects a lot.
import shutil
# Show the currently installed version of TensorFlow
print("Tensorflow version: {}".format(tf.__version__))

Tensorflow version: 2.3.2


## Input source: CSV Local

In [4]:
# Import data locally only the fist 100 rows to support
# the creation of the processing functions

# TODO update path to bucket
path_to_csv = os.path.join(LOCAL_ROOT_DIR, INPUT_FILE)
data = pd.read_csv(path_to_csv, nrows=100)

In [5]:
data.columns

Index(['POSTED_BY', 'UNDER_CONSTRUCTION', 'RERA', 'BHK_NO.', 'BHK_OR_RK',
       'SQUARE_FT', 'READY_TO_MOVE', 'RESALE', 'ADDRESS', 'LONGITUDE',
       'LATITUDE', 'TARGET(PRICE_IN_LACS)'],
      dtype='object')

In [6]:
data.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS)
0,Owner,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0
1,Dealer,0,0,2,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0
2,Owner,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0
3,Owner,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5
4,Dealer,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5


In [7]:
# Convert csv in a bigquery like processor
data_bq_like = list(data.iloc[:5].to_dict("index").values()) # sample just the first 10 rows to test the processing functions

## Create raw data metadata

In [22]:
CATEGORICAL_FEATURE_NAMES_STRING = ['POSTED_BY', 'BHK_OR_RK', 'ADDRESS']
#CATEGORICAL_FEATURE_MAX_VALUES = [24]
CATEGORICAL_FEATURE_NAMES_INT = ['UNDER_CONSTRUCTION', 'RERA',
                                 'READY_TO_MOVE', 'RESALE']

NUMERIC_FEATURE_NAMES = ['BHK','SQUARE_FT', 'LONGITUDE','LATITUDE']
#BUCKET_FEATURE_BUCKET_COUNT = [100]

TARGET_FEATURE_NAME = "TARGET_PRICE"

In [23]:
def create_raw_metadata():
    
    raw_data_schema = {}
    
 
    # target feature schema
    raw_data_schema[TARGET_FEATURE_NAME]= dataset_schema.ColumnSchema(
            tf.float32, [], dataset_schema.FixedColumnRepresentation())
    
    # categorical string features schema
    raw_data_schema.update({column_name : dataset_schema.ColumnSchema(
        tf.string, [], dataset_schema.FixedColumnRepresentation())
                            for column_name in CATEGORICAL_FEATURE_NAMES_STRING})
    
    # categorical string features schema
    raw_data_schema.update({column_name : dataset_schema.ColumnSchema(
        tf.int64, [], dataset_schema.FixedColumnRepresentation())
                            for column_name in CATEGORICAL_FEATURE_NAMES_INT})
    
    # numerical features schema
    raw_data_schema.update({ column_name : dataset_schema.ColumnSchema(
        tf.float32, [], dataset_schema.FixedColumnRepresentation())
                            for column_name in NUMERIC_FEATURE_NAMES})
    
    # create dataset_metadata given raw_schema
    raw_data_metadata = dataset_metadata.DatasetMetadata(
        dataset_schema.Schema(raw_data_schema))
    
    return raw_data_metadata

## Input source: BigQuery

In [25]:
from google.cloud import bigquery

def create_query(phase, hash_column, table_name, EVERY_N=None):
    """Creates a query with the proper splits.
    Args:
        phase: str, train, valid.
        hash_column: str, is the column that we will use to perform the split
        EVERY_N: int, take an example EVERY_N rows.

    Returns:
        Query string with the proper splits.
    """
    base_query = """
    SELECT POSTED_BY,
    UNDER_CONSTRUCTION, 
    RERA, 
    BHK_NO_+0.0 AS BHK,
    BHK_OR_RK,
    SQUARE_FT,
    READY_TO_MOVE,
    RESALE,
    ADDRESS,
    LONGITUDE,
    LATITUDE,
    TARGET_PRICE_IN_LACS_ AS TARGET_PRICE,
    FROM {0}
    """.format(table_name)
    
    if EVERY_N is None:
        if phase=="train":
            # training
            query = """{0} WHERE MOD(ABS(FARM_FINGERPRINT(CAST({1} AS STRING))),
            10) < 8""".format(base_query, hash_column)
        else:
            query = """{0} WHERE MOD(ABS(FARM_FINGERPRINT(CAST({1} AS STRING))),
            10) >= 8""".format(base_query, hash_column)
    else:
        query = """{0} WHERE MOD(ABS(FARM_FINGERPRINT(CAST({1} AS STRING))),
        {2})) = 2""".format(base_query,hash_column, EVERY_N)
        # random split WHERE RAND() < 0.8

    return query

In [26]:
# DEBUG validation of the BigQuery function
query = create_query("valid", "ADDRESS", "houseprice_data.train")
df_validation = bigquery.Client().query(query).to_dataframe()
# `head()` function is used to get the first n rows of dataframe
display(df_validation.head())

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET_PRICE
0,Dealer,1,1,2.0,BHK,985.291154,0,1,"Moshi,Pune",18.669919,73.853723,48.9
1,Dealer,1,1,2.0,BHK,715.059588,0,1,"Moshi,Pune",18.669919,73.853723,33.0
2,Dealer,1,1,2.0,BHK,1057.195953,0,1,"Moshi,Pune",18.669919,73.853723,51.2
3,Dealer,1,0,1.0,BHK,592.800528,0,1,"Moshi,Pune",18.669919,73.853723,35.9
4,Dealer,1,1,2.0,BHK,988.175676,0,1,"Moshi,Pune",18.669919,73.853723,46.8


In [27]:
def is_valid(inputs):
    """Check to make sure the inputs are valid.
    Args:
        inputs: dict, dictionary of TableRow data from BigQuery.

    Returns:
        True if the inputs are valid and False if they are not.
    """
    try:
        underconstruction = inputs["UNDER_CONSTRUCTION"]
        return underconstruction >= 0 and underconstruction  <2
    except:
        return False

In [28]:
def preprocess_tft(inputs):
    """
    Preprocess the features and add engineered features with tf transform.

    Args:
        dict, dictionary of TableRow data from BigQuery.

    Returns:
        Dictionary of preprocessed data after scaling and feature engineering.
    """
    # create a new dictionary that will contain only those relevan features selected
    result = {}
    # Report the label in the processed data set
    result["target_price"] = tf.identity(inputs["TARGET_PRICE"])
    
    # Build a vocobulary
    result["address"] = tft.compute_and_apply_vocabulary(inputs["ADDRESS"]) #remove the address and leave the city
    result["posted_by"] = tft.compute_and_apply_vocabulary(inputs["POSTED_BY"])
    result["bhk_or_rk"] = tft.compute_and_apply_vocabulary(inputs["BHK_OR_RK"])
    
    # Keep the  categorical int features untouch
    result["under_construction"] = tf.identity(inputs["UNDER_CONSTRUCTION"])
    result["rera"] = tf.identity(inputs["RERA"])
    result["ready_to_move"] = tf.identity(inputs["READY_TO_MOVE"])
    result["resale"] = tf.identity(inputs["RESALE"])

    # Scaling numeric values
    result['bhk_no'] = (tft.scale_to_0_1(inputs['BHK']))
    result['square_ft'] = (tft.scale_to_0_1(inputs['SQUARE_FT']))

    # Bucketize
    result['longitude_bucket'] = tft.bucketize(inputs["LONGITUDE"], num_buckets=100)
    result['latitude_bucket'] = tft.bucketize(inputs["LATITUDE"], num_buckets=100)
    
    
    # Engineered features
    # basic implementation of a feature created to (potentially) improve the prediction
    result["avg_room_sq.ft"] = tft.scale_to_0_1(inputs['SQUARE_FT'] / inputs['BHK'])
    
    return result

## Process function Apache Beam

In [29]:
def preprocess(in_test_mode):
    """Sets up preprocess pipeline.

    Args:
        in_test_mode: bool, False to launch DataFlow job, True to run locally.
    """
    import os
    import os.path
    import tempfile
    import datetime
    from apache_beam.io import tfrecordio
    from tensorflow_transform.coders import example_proto_coder
    from tensorflow_transform.tf_metadata import dataset_metadata
    from tensorflow_transform.tf_metadata import dataset_schema
    from tensorflow_transform.beam import tft_beam_io
    from tensorflow_transform.beam import impl as beam_impl
    from tensorflow_transform.beam.tft_beam_io import transform_fn_io

    job_name = 'house_price_tftprocessing' + '-'
    job_name += datetime.datetime.now().strftime('%y%m%d-%H%M%S')
    if in_test_mode:
        import shutil
        print('Launching local job ...')
        #OUTPUT_DIR = './processing_tft' #local saving
        OUTPUT_DIR = 'gs://{0}/processing_tft/'.format(BUCKET) # bucket saving
        shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
        EVERY_N = None
    else:
        print('Launching Dataflow job {} ...'.format(job_name))
        OUTPUT_DIR = 'gs://{0}/processing_tft/'.format(BUCKET)
        import subprocess
        subprocess.call('gsutil rm -r {}'.format(OUTPUT_DIR).split())
        EVERY_N = None

    options = {
        'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
        'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
        'job_name': job_name,
        'project': PROJECT,
        'num_workers': 1,
        'max_num_workers': 1,
        'teardown_policy': 'TEARDOWN_ALWAYS',
        'no_save_main_session': True,
        'direct_num_workers': 1,
        'extra_packages': ['tensorflow_transform-0.24.0-py3-none-any.whl']
        }

    opts = beam.pipeline.PipelineOptions(flags=[], **options)
    if in_test_mode:
        RUNNER = 'DirectRunner'
    else:
        RUNNER = 'DataflowRunner'

    # Set up raw data metadata
    raw_data_metadata = create_raw_metadata()

    # Run Beam
    with beam.Pipeline(RUNNER, options=opts) as p:
        with beam_impl.Context(temp_dir=os.path.join(OUTPUT_DIR, 'tmp')):
            # Save the raw data metadata
            (raw_data_metadata |
                'WriteInputMetadata' >> tft_beam_io.WriteMetadata(
                    os.path.join(
                        OUTPUT_DIR, 'metadata/rawdata_metadata'), pipeline=p))

            # Read training data from bigquery and filter rows
            raw_data = (p 
                        | 'Read train set' >> beam.io.Read(beam.io.BigQuerySource(query=create_query("train", "ADDRESS", "houseprice_data.train"),
                                                                              use_standard_sql=True)) 
                        | 'Validate data' >> beam.Filter(is_valid))

            raw_dataset = (raw_data, raw_data_metadata)

            # Analyze and transform training data
            transformed_dataset, transform_fn = (raw_dataset 
                                                 | beam_impl.AnalyzeAndTransformDataset(preprocess_tft))
            
            transformed_data, transformed_metadata = transformed_dataset

            # Save transformed train data to disk in efficient tfrecord format
            transformed_data | 'WriteTrainData' >> tfrecordio.WriteToTFRecord(os.path.join(OUTPUT_DIR, 'train'),
                                                                              file_name_suffix='.gz',
                                                                              coder=example_proto_coder.ExampleProtoCoder(transformed_metadata.schema))

            
            # Read eval data from bigquery and filter rows
            raw_test_data = (p 
                             | 'Read eval set' >> beam.io.Read(beam.io.BigQuerySource(query=create_query("valid", "ADDRESS", "houseprice_data.train"),
                                                                                        use_standard_sql=True)) 
                             | 'eval_filter' >> beam.Filter(is_valid))

            raw_test_dataset = (raw_test_data, raw_data_metadata)

            # Transform eval data
            transformed_test_dataset = ((raw_test_dataset, transform_fn) | beam_impl.TransformDataset())
            transformed_test_data, _ = transformed_test_dataset

            # Save transformed train data to disk in efficient tfrecord format
            (transformed_test_data | 'WriteTestData' >> tfrecordio.WriteToTFRecord(os.path.join(OUTPUT_DIR, 'eval'), 
                                                                                   file_name_suffix='.gz',
                                                                                   coder=example_proto_coder.ExampleProtoCoder(transformed_metadata.schema)))

            # Save transformation function to disk for use at serving time
            (transform_fn | 'WriteTransformFn' >> transform_fn_io.WriteTransformFn(os.path.join(OUTPUT_DIR, 'metadata')))

In [30]:
# Change to False to run on DataFlow
preprocess(in_test_mode=True)

Launching local job ...


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:70: BeamDeprecationWarning: BigQuerySource is deprecated since 2.25.0. Use ReadFromBigQuery instead.
/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1881: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(


Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


...

value: "\n\013\n\tConst_5:0\022/vocab_compute_and_apply_vocabulary_1_vocabulary"



value: "\n\013\n\tConst_7:0\022/vocab_compute_and_apply_vocabulary_2_vocabulary"



value: "\n\013\n\tConst_7:0\022/vocab_compute_and_apply_vocabulary_2_vocabulary"



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
